In [ ]:
!pip install dash dash-bootstrap-components pandas plotly
!pip install jupyter-dash
!pip install gunicorn

In [ ]:
import dash
from dash import html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px
from functools import lru_cache
import os
import json
from pyngrok import ngrok 
import nest_asyncio 
from io import StringIO 
import time



DATA_PATH = "df_joined.csv"   

@lru_cache(maxsize=1)
def load_data():
    if not os.path.exists(DATA_PATH):
        raise FileNotFoundError(f"'{DATA_PATH}' not found. Please upload this file to Colab.")
    return pd.read_csv(DATA_PATH, low_memory=False)

@lru_cache(maxsize=1) 
def load_metadata():
    try:
        df = load_data()
    except FileNotFoundError:
        return {
            "boroughs": [], "years": [], "vehicle_types": [], 
            "factors": [], "injuries": []
        }

    df = load_data()
    # Ensure all columns exist before accessing them
    boroughs = sorted(df["BOROUGH"].dropna().unique().tolist()) if "BOROUGH" in df else []
    years = sorted(pd.to_datetime(df["CRASH_DATE"], errors="coerce").dt.year.dropna().astype(int).unique().tolist()) if "CRASH_DATE" in df else []
    vehicle = sorted(df["VEHICLE TYPE CODE 1"].dropna().unique().tolist()) if "VEHICLE TYPE CODE 1" in df else []
    factors = sorted(df["CONTRIBUTING FACTOR VEHICLE 1"].dropna().unique().tolist()) if "CONTRIBUTING FACTOR VEHICLE 1" in df else []
    injuries = sorted(df["PERSON_INJURY"].dropna().unique().tolist()) if "PERSON_INJURY" in df else []

    return {
        "boroughs": boroughs,
        "years": years,
        "vehicle_types": vehicle,
        "factors": factors,
        "injuries": injuries
    }

def parse_search_query(search_query, metadata):
    """Parses a natural language query and extracts filter values."""
    if not search_query or not search_query.strip():
        return None 

    s = search_query.lower()
    parsed_filters = {
        "boroughs": [], "years": [], "injuries": []
    }
    
    # 1. Boroughs
    for b in metadata["boroughs"]:
        if b.lower() in s:
            parsed_filters["boroughs"].append(b)
            
    # 2. Years
    for year in metadata["years"]:
        if str(year) in s:
            parsed_filters["years"].append(year)
            
    # 3. Injuries (PERSON_INJURY)
    injury_keywords = {
        "pedestrian": ["PEDESTRIAN"], 
        "cyclist": ["BICYCLIST"],
        "motorist": ["PASSENGER", "DRIVER"],
        "killed": ["KILLED"],
        "injured": ["INJURED"]
    }
    for keyword, values in injury_keywords.items():
        if keyword in s:
            for val in values:
                if val in metadata["injuries"]:
                    parsed_filters["injuries"].append(val)
    
    if any(parsed_filters.values()):
        return parsed_filters
    return None

def apply_filters(df, boroughs, years, vehicles, factors, injuries):
    """Applies filters based ONLY on the current state of the dropdowns."""
    out = df.copy()

    if boroughs and "BOROUGH" in out:
        out = out[out["BOROUGH"].isin(boroughs)]
    if years and "CRASH_DATE" in out:
        out["CRASH_DATE"] = pd.to_datetime(out["CRASH_DATE"], errors="coerce")
        out = out[out["CRASH_DATE"].dt.year.isin(years)]
    if vehicles and "VEHICLE TYPE CODE 1" in out:
        out = out[out["VEHICLE TYPE CODE 1"].isin(vehicles)]
    if factors and "CONTRIBUTING FACTOR VEHICLE 1" in out:
        out = out[out["CONTRIBUTING FACTOR VEHICLE 1"].isin(factors)]
    if injuries and "PERSON_INJURY" in out:
        out = out[out["PERSON_INJURY"].isin(injuries)]

    return out

# Convert DF to JSON for dcc.Store
def df_to_json(df):
    return df.to_json(date_format="iso", orient="split")

def json_to_df(js):
    return pd.read_json(StringIO(js), orient="split")